This python program asks for a given cryptocurrency through the Twitter API in order to gather 100 most recents tweets about the cryptocurrency of interest. The gathered data is then use to assess sentiment by assigning a magnitude value for each tweet which are all totaled to assess the overall market, trend sentiment regarding the crypto in question.

Magnitude, sentiment, is calculated considering metrics such as likes, retweets and the account's metrics of the user who published the tweet, verified status or no , number of followrs. The sentiment is noted with the help of a complex analysis that gives ponderation points to the different keywords included in the tweet. This will rely on NLP, Natural Language Processing.

In [19]:
pip install TextBlob

You should consider upgrading via the 'c:\Users\Fluffy\AppData\Local\Programs\PythonCodingPack\python.exe -m pip install --upgrade pip' command.


In [20]:
import tweepy as twp
from textblob import TextBlob
from dotenv import load_dotenv
import os

import config

In [21]:
#Loading Twitter credentials 

API_KEY = config.api_key
API_KEY_SECRET = config.api_key_secret
BEARER_TOKEN = config.bearer_token
ACCESS_TOKEN = config.access_token
ACCESS_TOKEN_SECRET = config.access_token_secret


In [22]:
auth = twp.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = twp.API(auth)

Connotation, Polarity => Sentiment Score.

In [23]:
#Functions
def get_polarity(tweet):
    return TextBlob(tweet).sentiment.get_polarity

def get_sentiment(score):
    if score < 0 :
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    elif score > 0:
        return 'Positive'

#Check if account is verified and assess number of followers
def find_account_value(tweet):
    value = 1
    ver = tweet.user.verified
    if(ver == True):
        value *= 1.10
    elif(tweet.user.followers_count > 100000):
        value *= 1.05
    else:
        value += 0.05
    return value

#Function that gives a score/ value to each tweet

def find_tweet_value(list_of_tweets, tweet):

    tweet_likes = tweet.favorite_count
    tweet_rts = tweet.retweet_count

    total_likes = 0
    total_rts = 0
    for tweet in list_of_tweets:
        total_likes += tweet.favorite_count
        total_rts += tweet.retweet_count

    avg_likes = total_likes / len(list_of_tweets)
    avg_rts = total_rts / len(list_of_tweets)

    value = 5

    #Checking LIKES
    if tweet_likes > avg_likes:
        value += tweet_likes / avg_likes
    elif tweet_likes < avg_likes:
        value -= tweet_likes / avg_likes

    #Checking RETWEETS
    if tweet_rts > avg_rts:
        value += tweet_rts / avg_rts
    elif tweet_rts < avg_rts:
        value -= tweet_rts /avg_rts
    return value

Generating Magnitude with the help of Polarity & Sentiment

In [24]:
def magnitude(list_of_tweets, tweet):
    pol = get_polarity(tweet.text)
    sen = get_sentiment(pol)

    magni = 0

    if (sen == 'Negative'):
        magni = find_account_value(tweet) + find_tweet_value(list_of_tweets, tweet) * -1
    elif(sen == 'Neutral'):
        magni = 0
    else:
        magni = find_account_value(tweet) + find_tweet_value(list_of_tweets, tweet)
    return magni

In [25]:
def get_connotation(list_of_tweets):
    connot = 0
    for tweet in list_of_tweets:
        con += magnitude(list_of_tweets, tweet)
    return connot

Looping through all the relevant tweets and printing the information regarding each tweet.

In [26]:
def print_tweets(list_of_tweets):
    for tweet in list_of_tweets:
        tweet_value = find_tweet_value(list_of_tweets, tweet)
        acc_value = find_account_value(tweet)
        magni_value = magnitude(list_of_tweets, tweet)
        print(f"Verified: {tweet.user.verified}")
        print(f"{tweet.user.screen.name} tweeted: ")
        print("---------")
        print(tweet.text)
        print("---------")
        print(f"# of likes: {tweet.favorite.count}")
        print(f"# of Retweets: {tweet.retweet.count}")
        print(f'Tweet Value: {tweet_value}')
        print(f"Account Value: {acc_value}")
        print(f"Tweet Magnitude: {magni_value}")

        #Sentiment
        pol = get_polarity(tweet.text)
        sen = get_sentiment(pol)
        print(f"Sentiment: {sen}")
        print("--------------------------------------------------------------------------------------")

Main Function to input crypto of interest and run analysis.

In [30]:
def main():
    search = input("Which coin would you like to analyze?")
    search_terms = search + " -filter:retweets"

    search_tweets = api.search_tweets(q = search_terms, lang = 'en', count = 100000, result_type = "mixed")

    print(search.upper())
    print(f"Number of tweets found: {len(search_tweets)}")
    print("-----------------------------------------------------------------------------------------------")

    print_all_tweets(search_tweets)

    connotation = get_connotation(search_tweets)g
    print(f"Current Connotation: {connotation}")



In [32]:
main()

KeyboardInterrupt: Interrupted by user